# 1.0 Notebook Parameters

Please specify the following variables:
1. `project_directory`: your project directory path
2. `playerID_list`: the list of players (i.e. skill models) you want to consider in your analysis; usually use one base model and scale it with different epsilons
3. `epsilon_list`: the list of execution error multiplers (epsilon) that you want to consider in your analysis.

In [31]:

# Change this to the project directory path for your computer  
project_directory = '/Users/rachael/Desktop/darts-thesis'

# Set the ID(s) of the player skill models you want to use 
    # 10 - symmetric DB skill model that we were using up until now 
    # ? - "average pro" skill model generated by H & W 
    
playerID_list = [10]
#epsilon_list = [1,2,3,4,5,6,7,8,9]
epsilon_list = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20,32,64,128,25,50,100]

# 2.0 Notebook Setup

Import libraries and run prerequisites.

In [32]:
# Import and configure modules from this project

import os

os.chdir(project_directory)

import function_helpers_singleplayer as h 
import function_init_board as fb 

data_parameter_dir = fb.data_parameter_dir
result_dir = './result/singlegame_results'  

In [ ]:
# Import and configure external modules 

import pandas as pd

import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(linewidth=300)
np.set_printoptions(threshold=300)

import torch
torch.set_printoptions(precision=4)
torch.set_printoptions(linewidth=300)
torch.set_printoptions(threshold=300)

# This makes the code output easier to read
# Can comment out if you want to see warnings 
import warnings
warnings.filterwarnings("ignore")

In [34]:
# Integrate Skill Models for Desired Players to Get Transition Probabilities

for player in playerID_list:

    for epsilon in epsilon_list: 

        # Path of data file 
        data_path = project_directory + '/data_parameter/player_gaussin_fit/grid_full/e{}_player{}_gaussin_prob_grid.pkl'.format(epsilon,player)
        
    # Generate if it does not yet exist
    if not(os.path.exists(data_path)):

        h.evaluate_score_probability([player],epsilon=epsilon,f_density_grid_pixel_per_mm=0.5)

In [35]:
# Reconfigure and Save Transition Probabilities

#################################
## Token Transition Probabilities 
#################################

# Path of data file 
data_path = project_directory + '/data_parameter/player_gaussin_fit/grid_custom_tokens/player10_gaussin_prob_grid_custom_tokens.pkl'

# Generate if it does not yet exist
if not(os.path.exists(data_path)):
    h.save_aiming_grid_custom_tokens()

#####################################
## No-Token Transition Probabilities 
#####################################

for player in playerID_list:

    for epsilon in epsilon_list: 

        # Path of data file 
        path = project_directory + '/data_parameter/player_gaussin_fit/grid_custom_no_tokens/player{}_e{}_gaussin_prob_grid_custom_no_tokens.pkl'.format(player,epsilon)

        # Generate if it does not yet exist
        if not(os.path.exists(data_path)):

            h.save_aiming_grid_custom_no_tokens([player],epsilon=epsilon)

# 3.0 Single Player Sandbox

Can alter the parameters and examine results for a single player here.

In [36]:
player_num = 10 
epsilon = 1
name_pa = 'player{}'.format(player_num)

[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = h.load_aiming_grid(name_pa, epsilon=epsilon, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

prob_grid_doublescore_dic = None
prob_grid_doublescore_dic_t = None

[optimal_value, optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = 9, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)

print('------------------------------------------------------------------')
print('Optimal Values: ')
print(optimal_value)
print('------------------------------------------------------------------')
print('Optimal Policy}: ')
print(optimal_action_index)

load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e1_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/t_gaussin_prob_grid_custom_tokens.pkl


/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:718: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:718: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:756: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:756: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


------------------------------------------------------------------
Optimal Values: 
[[ 0.      0.      2.8875 ... 15.796  15.7386 15.854 ]
 [ 0.      0.      1.     ... 14.031  14.0166 14.0347]
 [ 0.      0.      1.     ... 13.3518 13.3227 13.3554]
 ...
 [ 0.      0.      1.     ... 10.1522 10.1477 10.1718]
 [ 0.      0.      1.     ...  9.5936  9.5782  9.5959]
 [ 0.      0.      1.     ...  9.      9.      9.    ]]
------------------------------------------------------------------
Optimal Policy}: 
[[ -1  -1  55 ...  10  10  10]
 [ -1  -1 752 ...  10  10  10]
 [ -1  -1 752 ...  10  10  10]
 ...
 [ -1  -1 752 ...  10  10  10]
 [ -1  -1 752 ... 406 274 771]
 [ -1  -1 752 ... 750 750 750]]


# 4.0 Generate Dataset


In [37]:
# Helper function to deconstruct the solved MDPs and reformat as a dataframe 
def create_result_dataframes(player, optimal_value,optimal_action_index):
    
    col_names = ['player','epsilon','tokens','score','optimal_value','optimal_policy']
    df_list = []
    
    for t in range(optimal_value.shape[0]):
        for s in range(optimal_value.shape[1]):
            df_list.append([player,epsilon,t,s,optimal_value[t][s],optimal_action_index[t][s]])

    df = pd.DataFrame(df_list,columns=col_names)
    
    return df 


In [38]:
col_names = ['player','epsilon','tokens','score','optimal_value','optimal_policy']
df = pd.DataFrame(columns=col_names)

for player_num in playerID_list:

    name_pa = 'player{}'.format(player_num)

    [aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')
    prob_grid_doublescore_dic = None
    prob_grid_doublescore_dic_t = None

    for epsilon in epsilon_list:
        [aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = h.load_aiming_grid(name_pa, epsilon=epsilon, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
        [optimal_value, optimal_action_index] = h.solve_dp_noturn_tokens(aiming_grid, prob_grid_normalscore_nt, prob_grid_normalscore_t, tokens = 9, prob_grid_doublescore=prob_grid_doublescore_nt, prob_grid_bullscore=prob_grid_bullscore_nt, prob_grid_doublescore_dic=None, prob_grid_doublescore_t=prob_grid_doublescore_t, prob_grid_bullscore_t=prob_grid_bullscore_t, prob_grid_doublescore_dic_t=None)
        df = pd.concat([df,create_result_dataframes(player_num,optimal_value,optimal_action_index)])
        df = df.reset_index().drop(['index'],axis=1)


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/t_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e1_gaussin_prob_grid_custom_no_tokens.pkl


/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:718: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:718: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:756: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:756: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e2_gaussin_prob_grid_custom_no_tokens.pkl


/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:718: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:756: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e3_gaussin_prob_grid_custom_no_tokens.pkl


...

/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:718: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:756: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e50_gaussin_prob_grid_custom_no_tokens.pkl


/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:718: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:756: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e100_gaussin_prob_grid_custom_no_tokens.pkl


/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:718: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/function_helpers_singleplayer.py:756: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate


In [39]:
# Save dataset 
result_path = result_dir + '/player10_noturn_results.csv'
df.to_csv(result_path)